# HWcards (HNU CE 2022년 2학기 객체지향프로그래밍 과제)

- 이름: 배정훈
- 학번: 20190580
  

Playing Cards로 하는 SPOONS같은 게임을 만들기 위한 준비작업을 하는 과제다.

- `Deck` 인터페이스를 구현하는 `GameDeck` 클래스 작성
  - `GameDeck`에 포함된 `CardItem`객체는 up 속성을 `false`로 하여 카드 뒷면이 보이게
- `Player` 인터페이스를 구현하는 두 종류의 클래스 `HumanPlayer`와 `ComputerPlayer`를 작성
  - `HumanPlayer`는
    - `hand`에 든 `CardItem`객체를 `up` 속성을 `true`로 하여 카드 앞면이 드러나 보이게
    - `put`메소드에서 표준입력으로 0부터 시작하는 정수 형태의 인덱스 값을 사용자에게 입력받아 `hand`의 해당 인덱스 위치의 카드를 내려놓도록 작성
  - `ComputerPlayer`는
    - `hand`에 든 `CardItem`객체의 `up` 속성을 `false`로 하여 카드 뒷면이 보이게
    - 일단 이 과제에서는 `ComputerPlayer`는 무조건 `hand`의 첫 카드(즉 인덱스 0)를 내려놓도록 `put`메소드 작성
- `GameDeck`, `HumanPlayer`, `ComputerPlayer` 모두 보기좋게 보여주기 위해 `ToHTML` 인터페이스도 구현하라 

In [11]:
enum class Symbol { SPADE, HEART, DIA, CLUB; } // 모양

enum class Card(val face: String, val back: String = "🂠") { // 카드의 종류
    SA("🂡"),S2("🂢"),S3("🂣"),S4("🂤"),S5("🂥"),S6("🂦"),
    S7("🂧"),S8("🂨"),S9("🂩"),S0("🂪"),SJ("🂫"),SQ("🂭"),SK("🂮"),
    HA("🂱"),H2("🂲"),H3("🂳"),H4("🂴"),H5("🂵"),H6("🂶"),
    H7("🂷"),H8("🂸"),H9("🂹"),H0("🂺"),HJ("🂻"),HQ("🂽"),HK("🂾"),
    DA("🃁"),D2("🃂"),D3("🃃"),D4("🃄"),D5("🃅"),D6("🃆"),
    D7("🃇"),D8("🃈"),D9("🃉"),D0("🃊"),DJ("🃋"),DQ("🃍"),DK("🃎"),
    CA("🃑"),C2("🃒"),C3("🃓"),C4("🃔"),C5("🃕"),C6("🃖"),
    C7("🃗"),C8("🃘"),C9("🃙"),C0("🃚"),CJ("🃛"),CQ("🃝"),CK("🃞");
    
    fun symbol() = when(this) {
        in SA..SK -> Symbol.SPADE
        in HA..HK -> Symbol.HEART
        in DA..DK -> Symbol.DIA
        else      -> Symbol.CLUB // 나머지 경우는 
    }
    
    fun color() = when(symbol()) {
        Symbol.SPADE -> "black"
        Symbol.HEART -> "red"
        Symbol.DIA   -> "red"
        Symbol.CLUB  -> "black"
    }
    
    fun rank() = this.ordinal % 13 + 1 // A가 1, 숫자는 숫자값, J,Q,K는 11,12,13
}

In [12]:
data class CardItem(val card: Card, var up: Boolean = false) { // 카드 한장
    fun symbol() = card.symbol()
    fun color() = card.color()
    fun rank() = card.rank()
    fun show() = if(up) card.face else card.back
}

In [13]:
object MyUtil {
    fun readLn(): String {
        try {
            return readln()
        } catch (e: Exception) {
            val msg = e.message
            if (msg != null) {
                val curr = msg.split('\n').last()
                val prefix = "Current input: {\"status\":\"ok\",\"value\":"
                if ( curr.startsWith(prefix) ) {
                    return curr.drop( prefix.length + 1 ).dropLast(1 + 1)
                }
            }
            throw e
        }
    }
    fun readNonNegativeIntUntilSuccess(): Int {
        do {
             val z = this.readLn().toIntOrNull()
             if (z != null && z >= 0) return z
        } while (true)
    }
    val style = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Noto+Sans+Symbols+2&display=swap');
.card {
    display: inline-block; 
    font-size: 60pt; height: 62pt;
    font-family: "Noto Sans Symbols 2", sans-serif;
    background: rgba(255,255,255, 1) !important;
}
.deck { margin-right:-32pt; }
</style>
"""
}
}

In [14]:
HTML(
    Card.values().toList().shuffled()
        .map { CardItem(it,true) }
        .fold(MyUtil.style) { acc, c -> acc +
"""<span class='card deck'
        style='color: ${if (c.up) c.color() else "black"};'>${c.show()}</span>"""
        }
)

🂻 🂫 🃘 🃔 🃋 🃛 🃇 🃓 🃆 🃒 🃎 🃁 🃞 🃂 🃕 🂴 🂩 🃗 🂽 🂤 🂱 🃈 🂦 🃊 🃙 🂢 🂵 🃅 🂹 🂲 🃄 🂶 🂷 🂣 🃚 🂨 🃍 🂮 🂾 🂸 🃝 🃉 🂪 🂡 🃖 🂥 🂭 🂳 🃃 🂺 🃑 🂧

In [15]:
interface Deck {
    fun size(): Int
    fun draw(): CardItem // 덱에서 카드를 뽑기
    fun put(c: CardItem) // 덱에 카드 c를 놓기
}

interface Player {
    val hand: MutableList<CardItem> // 이것도 가능!!
    fun draw(d: Deck) // 덱 d로부터 카드 하나 뽑기
    fun put(d: Deck)  // 카드 하나를 덱 d에 놓기
}

interface ToHTML {
    fun toHTML(): String // 해당 객체를 표현하는 HTML 소스코드 문자열 생성
}

In [17]:

class GameDeck() : Deck, ToHTML
{
    val d_list : MutableList<CardItem>
    init
    {
        d_list = Card.values().toList().shuffled().map { CardItem(it, false) }. shuffled().toMutableList()
    }
    
    override fun size() = d_list.size
    override fun draw() : CardItem{
        val t_deck = d_list[d_list.size - 1]
        d_list.removeAt(d_list.size - 1)
        return t_deck
    }

    override fun put(c : CardItem){
        d_list.add(c)
    }

    override fun toHTML(): String{
        val m: String = d_list.fold(MyUtil.style)
        { 
            acc, c -> acc + """<span class='card deck' style='color: ${if (c.up) c.color() else "black"};'>${c.show()}</span>"""
        }
        return m
    }
    

}

In [43]:
open class HumanPlayer() : Player, ToHTML{
    override val hand: MutableList<CardItem> = mutableListOf();
    override fun draw(d: Deck){
        hand.add(CardItem(d.draw().card, true));
    }
    override fun put(d: Deck){
        var num: Int = MyUtil.readNonNegativeIntUntilSuccess()
        d.put(hand[num]);
        hand.removeAt(num);

    }
    override fun toHTML(): String{
        val m: String = hand.fold(MyUtil.style)
        { 
            acc, c -> acc + """<span class='card deck' style='color: ${if (c.up) c.color() else "black"};'>${c.show()}</span>"""
        }
        return m
    }
}

In [23]:
class ComputerPlayer : Player, ToHTML{
    override val hand: MutableList<CardItem> = mutableListOf();
    override fun draw(d: Deck){
        hand.add(CardItem(d.draw().card, false));
    }
    override fun put(d: Deck){
        d.put(hand[0]);
        hand.removeAt(0);

    }
    override fun toHTML(): String{
        val m: String = hand.fold(MyUtil.style)
        { 
            acc, c -> acc + """<span class='card deck' style='color: ${if (c.up) c.color() else "black"};'>${c.show()}</span>"""
        }
        return m
    }
}

In [27]:
var deck =GameDeck()
HTML(deck.toHTML())

🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠

In [25]:
p_deck.size()

52

In [26]:
var c_dec = ComputerPlayer()
var p_dec = HumanPlayer()

In [31]:
p_dec.draw(deck)
HTML(p_dec.toHTML())

🃊 🂽 🂧 🃝

In [33]:
deck.size()

48

In [34]:
p_dec.put(deck)
HTML(p_dec.toHTML())

stdin: 2


🃊 🂽 🃝

In [35]:
deck.size()

49

In [38]:
c_dec.draw(deck)
HTML(c_dec.toHTML())

🂠 🂠

In [39]:
deck.size()

47

In [40]:
c_dec.put(deck)
HTML(c_dec.toHTML())

🂠

In [41]:
deck.size()

48